# Lab 9 - DEFI(Uniswap V3)

## 4. Deploy Pool

In [ ]:
artifacts = {
    WETH9: require("./build/WETH9.json"),
    UniswapV3Factory: require("./build/UniswapV3Factory.json"),
    UniswapV3Pool: require("./build/UniswapV3Pool.json"),
    NFTDescriptor: require("./build/NFTDescriptor.json"),
    NonfungibleTokenPositionDescriptor: require("./build/NonfungibleTokenPositionDescriptor.json"),
    NonfungiblePositionManager: require("./build/NonfungiblePositionManager.json"),
    ERC20: require("./artifacts/contracts/TutorialERC20.sol/TutorialERC20.json"),
};

In [ ]:
contracts = {
  WETH: '0x78E6935fD0D9b57B1Bc20204BA769bCEd249012E',
  UNISWAPV3_FACTORY: '0xfF2868E36dEfdD0180DFaFd4c234B9c2588a1887',
  NONFUNGIBLE_POSITION_MANAGER: '0xFB04DA8DCc30fbD00FD953ADE7414953Ccc96145',
  TOKEN_0: '0x3cE05DC47cCBd7c62E6dC6f1d6a08493B18B7992',
  TOKEN_1: '0xff64fbE6824cC5CA450FD4Ca22c249a6d375028f'
};


In [ ]:
new Promise(async (res, rej) => {
    const { ethers, BigNumber, Contract } = require("ethers");
    const provider = new ethers.providers.JsonRpcProvider("http://localhost:8545");
    const account = provider.getSigner();

    reserve0 = ethers.utils.parseEther("1000");
    reserve1 = ethers.utils.parseEther("1000");

    // Approve Withdrawal by PositionManager
    const tokenA = new Contract(contracts.TOKEN_0,artifacts.ERC20.abi)
    await tokenA
        .connect(account)
        .approve(contracts.NONFUNGIBLE_POSITION_MANAGER,reserve0);
    const tokenB = new Contract(contracts.TOKEN_1, artifacts.ERC20.abi);
    await tokenB
        .connect(account)
        .approve(contracts.NONFUNGIBLE_POSITION_MANAGER, reserve1);

    // Create price function
    const bn = require("bignumber.js");
    bn.config({ EXPONENTIAL_AT: 999999, DECIMAL_PLACES: 40 });
    function encodePriceSqrt(reserve1, reserve0) {
        return BigNumber.from(
            new bn(reserve1.toString())
                .div(reserve0.toString())
                .sqrt()
                .multipliedBy(new bn(2).pow(96))
                .integerValue(3)
                .toString()
        );
    }

    const factory = new Contract(
        contracts.UNISWAPV3_FACTORY,
        artifacts.UniswapV3Factory.abi,
        provider
    );
    const nonfungiblePositionManager = new Contract(
        contracts.NONFUNGIBLE_POSITION_MANAGER,
        artifacts.NonfungiblePositionManager.abi,
        provider
    );

    // Deploy Pool
    tradingfee = 500;
    const price = encodePriceSqrt(1, 1);
    const response =
    await nonfungiblePositionManager
        .connect(account)
        .createAndInitializePoolIfNecessary(
            contracts.TOKEN_0,
            contracts.TOKEN_1,
            tradingfee,
            price,
            {
                gasLimit: 15000000,
            }
        );
    await response.wait();

    POOL = await factory.getPool(contracts.TOKEN_0, contracts.TOKEN_1, tradingfee);
    contracts = { ...contracts, POOL: POOL };
    console.log(contracts);

});
